In [2]:
import pandas as pd

### 2014 source file spec

In [3]:
# Define the file path
file_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\raw\kdkf_2014_raw.xlsx'

# Load the Excel file to get sheet names
xls = pd.ExcelFile(file_path)

# Define the years
previous_year = 2013
start_year = 2014
target_year = 2015

# Set the pandas option to display floating point numbers using the decimal format
pd.options.display.float_format = '{:.5f}'.format

### Master Loop

In [4]:
# Initialize an empty DataFrame for the master data
master_2014 = pd.DataFrame()

# Loop over each sheet in the Excel file
for sheet_name in xls.sheet_names:
    # Check if the sheet name starts with "HRM2_KT_"
    if sheet_name.endswith("HRM2"):
        # Load the sheet into a DataFrame, skipping the first three rows
        df = pd.read_excel(file_path, sheet_name=sheet_name, skiprows=0)

        # Merge the first row with the column headers
        df.columns = df.iloc[0, :2].tolist() + (df.columns[2:]).tolist()
        df = df.iloc[1:].reset_index(drop=True)

        # Update Headers of the df
        rename_dict = {
            'Rechnung': f'Rechnung {previous_year}',
            'Budget': f'Budget {start_year}',
            'Rechnung.1': f'Rechnung {start_year}',
            'Budget.1': f'Budget {target_year}'
        }
        df.rename(columns=rename_dict, inplace=True)

        # Rename the first column to 'HRM 2' and third column to "in 1000 CHF"
        df.columns.values[0] = 'HRM2-ID'
        df.columns.values[2] = 'HRM2-Bezeichnung'

        # Delete unecessary columns
        df.drop(df.columns[1], axis=1, inplace=True)
             
        # Add a new column with the DataFrame name (sheet name)
        df['Source'] = sheet_name
        df['Kanton'] = df['Source'].str.split(' HRM2').str[0]
        df['Year'] = 2014

        # Use pd.concat to add the DataFrame to the master DataFrame
        master_2014 = pd.concat([master_2014, df], ignore_index=True)

# Display the first few rows of the master DataFrame
master_2014.head()

,HRM2-ID,HRM2-Bezeichnung,Rechnung 2013,Budget 2014,Rechnung 2014,Budget 2015,Source,Kanton,Year,Compte,Compte.1
0,ERFOLGSRECHNUNG,NaN,NaN,def.,NaN,def.,ZH HRM2,ZH,2014,NaN,NaN
1,30,Personalaufwand,4727960.22320,4944121.85400,4958624.94034,5058273.57900,ZH HRM2,ZH,2014,NaN,NaN
2,31,Sach- und übriger Betriebsaufwand,2856763.26085,2972686.19200,2943690.29170,2976696.62825,ZH HRM2,ZH,2014,NaN,NaN
3,davon 314,baulicher und betrieblicher Unterhalt,207716.98300,213736.65000,214002.93191,209706.10000,ZH HRM2,ZH,2014,NaN,NaN
4,davon 3180,Wertberichtigungen auf Forderungen,3809.96198,916.60000,-3425.25279,897,ZH HRM2,ZH,2014,NaN,NaN


### Translating FR columns to DE

In [5]:
# Display rows where the column 'Kanton' is 'VD HRM2'
filtered_rows = master_2014[master_2014['Kanton'] == 'VD']

# Display the filtered rows
filtered_rows.head()

,HRM2-ID,HRM2-Bezeichnung,Rechnung 2013,Budget 2014,Rechnung 2014,Budget 2015,Source,Kanton,Year,Compte,Compte.1
3322,Compte de résultats,NaN,NaN,def.,NaN,NaN,VD HRM2,VD,2014,NaN,NaN
3323,30,Charges de personnel,NaN,2419168.50000,NaN,2407976.70000,VD HRM2,VD,2014,NaN,2463959.26779
3324,31,Charges de biens et services et autres charges...,NaN,806909.60000,NaN,833525.50000,VD HRM2,VD,2014,NaN,872251.11242
3325,de cela 314,Gros entretien et entretien courant,NaN,62521.10000,NaN,73680.00000,VD HRM2,VD,2014,NaN,71818.69427
3326,de cela 3180,Réévaluations sur créances,NaN,80016.50000,NaN,100014.00000,VD HRM2,VD,2014,NaN,16919.89261


In [6]:
# 1. Update 'Rechnung 2013' based on 'Compte'
master_2014['Rechnung 2013'] = master_2014.apply(
    lambda row: row['Compte'] if pd.isna(row['Rechnung 2013']) and not pd.isna(row['Compte']) else row['Rechnung 2013'],
    axis=1
)

# 2. Update 'Rechnung 2014' based on 'Compte.1'
master_2014['Rechnung 2014'] = master_2014.apply(
    lambda row: row['Compte.1'] if pd.isna(row['Rechnung 2014']) and not pd.isna(row['Compte.1']) else row['Rechnung 2014'],
    axis=1
)

# Display the first few rows of the updated DataFrame
master_2014.head()


,HRM2-ID,HRM2-Bezeichnung,Rechnung 2013,Budget 2014,Rechnung 2014,Budget 2015,Source,Kanton,Year,Compte,Compte.1
0,ERFOLGSRECHNUNG,NaN,NaN,def.,NaN,def.,ZH HRM2,ZH,2014,NaN,NaN
1,30,Personalaufwand,4727960.22320,4944121.85400,4958624.94034,5058273.57900,ZH HRM2,ZH,2014,NaN,NaN
2,31,Sach- und übriger Betriebsaufwand,2856763.26085,2972686.19200,2943690.29170,2976696.62825,ZH HRM2,ZH,2014,NaN,NaN
3,davon 314,baulicher und betrieblicher Unterhalt,207716.98300,213736.65000,214002.93191,209706.10000,ZH HRM2,ZH,2014,NaN,NaN
4,davon 3180,Wertberichtigungen auf Forderungen,3809.96198,916.60000,-3425.25279,897,ZH HRM2,ZH,2014,NaN,NaN


In [7]:
# Display rows where the column 'Kanton' is 'VD_HRM2'
filtered_rows = master_2014[master_2014['Kanton'] == 'VD HRM2']

# Display the filtered rows
filtered_rows.head()

,HRM2-ID,HRM2-Bezeichnung,Rechnung 2013,Budget 2014,Rechnung 2014,Budget 2015,Source,Kanton,Year,Compte,Compte.1


### Before saving the data, we drop non-numeric "HRM2-IDs", because they would cause issues with subsequent automated analysis...

In [8]:
# Drop rows where 'HRM-ID' values are non-numeric
master_2014 = master_2014[pd.to_numeric(master_2014['HRM2-ID'], errors='coerce').notna()]

# Display the first few rows of the updated DataFrame
len(master_2014)

2184

In [9]:
# Define the path where the CSV file will be saved
output_path = r'C:\Users\domin\Documents\GitHub\xai_budgeting\data\ready\kdkf_2014_data.csv'

# Save the DataFrame to a CSV file
master_2014.to_csv(output_path, index=False)
